# Homework 1 answers: Titantic dataset

In [1]:
from __future__ import division
import pandas as pd

# Load dataset into DataFrame
path = '/Users/Bronco/Classes/General_assembly/DAT_SF_13/homework/titanic.csv'

titanic_df = pd.read_csv(path, index_col = 'PassengerId')
titanic_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


####1.    How many passengers?

In [2]:
naboard = titanic_df.index.size
print 'There were {} passengers aboard.'.format(naboard)

There were 891 passengers aboard.


####2.    What is the overall survival rate?

In [3]:
nsurvived = titanic_df.Survived[titanic_df.Survived > 0].count()
survival_rate = nsurvived/naboard
print '{:.2%} of the people on board survived.'.format(survival_rate)

38.38% of the people on board survived.


####3.    How many male passengers were on board?

In [4]:
nmale = titanic_df.Sex[titanic_df.Sex == 'male'].count()
print 'There were {} male passengers on board.'.format(nmale)

There were 577 male passengers on board.


####4.    How many female passengers were on board?

In [5]:
nfemale = titanic_df.Sex[titanic_df.Sex == 'female'].count()
print 'There were {} female passengers on board.'.format(nfemale)

There were 314 female passengers on board.


####5.    What is the overall survival rate of male passengers?

In [6]:
nmale_survived = len(titanic_df[(titanic_df.Sex == 'male') & (titanic_df.Survived == 1)])
print '{:.2%} of the male passengers survived.'.format(nmale_survived/nmale)

18.89% of the male passengers survived.


####6.    What is the overall survival rate of female passengers?

In [7]:
nfemale_survived = len(titanic_df[(titanic_df.Sex == 'female') & (titanic_df.Survived == 1)])
print '{:.2%} of the female passengers survived.'.format(nfemale_survived/nfemale)

74.20% of the female passengers survived.


####7. What is the average age of all passengers on board?

In [8]:
mean_ages = titanic_df.Age.mean()
print 'The average age of passengers aboard the Titantic was {:0.1f}.'.format(mean_ages)

The average age of passengers aboard the Titantic was 29.7.


To calculate the average age I just used the pandas `.mean()` method, which calculates the mean of a series while dropping missing values. This is approximate since we don't know how the missing values might have affected the mean. Other ways to deal with the missing values are to fill them in with the mean calculated as above, which won't affect the mean, but might affect the median in future calculations. You could also try to guess the values from the other data in case there is a correllation with other metrics, such as class. If future calculations using the age can accept weights associated with the data you could give reported ages a weight of 1 and then guess the ages of the rest using a function that could give a probability that would serve as the weighting factor.

####8.    What is the average age of passengers that survived?

In [9]:
mean_age_survived = titanic_df[titanic_df.Survived == 1].Age.mean()
print 'The average age of passengers that survived is {:.1f}.'.format(mean_age_survived)

The average age of passengers that survived is 28.3.


####9.    What is the average age of passengers that died?

In [10]:
mean_age_died = titanic_df[~(titanic_df.Survived == 1)].Age.mean()
print 'The average age of passengers that survived is {:.1f}.'.format(mean_age_died)

The average age of passengers that survived is 30.6.


####10.    At this (early) point in our analysis, what might you infer about any patterns you are seeing?


It is pretty clear so far that women had a better chance of survival than men. I'm guessing that chivalry and traditional gender roles played a large part in this. Also, age may have been a factor, although it's hard to say how significant a role it played given the missing data. The difference is small so it could just be by chance. 

Randomly reshuffling who survived 10000 times and observing the liklihood of seeing a difference in ages as great as 28.3 - 30.6 could show whether this is a significant result:

In [11]:
import numpy as np

original_diff = mean_age_survived - mean_age_died

results = []

n = 10000

for i in xrange(n):
    idx_survived = np.random.choice(naboard, nsurvived, replace = False)
    mask_dead = np.zeros(naboard, bool)
    mask_dead[idx_survived] = True
    bs_mean_age_survived = titanic_df.Age.where(mask_dead).mean()
    bs_mean_age_died = titanic_df.Age.where(~mask_dead).mean()
    difference = bs_mean_age_survived - bs_mean_age_died
    results.append(difference)

results = pd.Series(results)
prob_bs = results[results < original_diff].count()/n

print('Reshuffling the data {} times with respect to who survived shows that\n'.format(n) +
      'the probability of observing a difference as great as {:.2f} years is {}'.format(original_diff,prob_bs))

Reshuffling the data 10000 times with respect to who survived shows that
the probability of observing a difference as great as -2.28 years is 0.0192


The above result does suggest that the small seemingly difference seen in ages between those that survived and died may in fact be significant using the p < 0.05 convention. It makes sense that age would be a factor in the harsh conditions of the North Atlantic.

####11.     How many passengers are in each of the three classes of service (e.g. First, Second, Third?)

In [12]:
classes = {1:'first', 2:'second', 3:'third'}
for i in range(1,4):
    num_passengers = len(titanic_df.Pclass[titanic_df.Pclass == i])
    print 'There were {} passengers in {} class.'.format(num_passengers,classes[i])

There were 216 passengers in first class.
There were 184 passengers in second class.
There were 491 passengers in third class.


####12.    What is the survival rate for passengers in each of the three classes of service?

In [13]:
for i in range(1,4):
    passengers = titanic_df.Pclass[titanic_df.Pclass == i]
    passengers_survived = passengers[titanic_df.Survived == 1]
    pct_survived = len(passengers_survived)/len(passengers)
    print 'There were {:.2%} passengers in {} class.'.format(pct_survived,classes[i])

There were 62.96% passengers in first class.
There were 47.28% passengers in second class.
There were 24.24% passengers in third class.


Given the previous result showing the effect of sex on survival rate it would be interesting to know how this breaks down by class:

In [14]:
for i in range(1,4):
    mpassengers = titanic_df.Pclass[(titanic_df.Pclass == i) & (titanic_df.Sex == 'male')]
    fpassengers = titanic_df.Pclass[(titanic_df.Pclass == i) & (titanic_df.Sex == 'female')]
    msurvived = mpassengers[titanic_df.Survived == 1]
    fsurvived = fpassengers[titanic_df.Survived == 1]
    pct_male = len(msurvived)/len(mpassengers)
    pct_female = len(fsurvived)/len(fpassengers)
    print 'In {} class {:.2%} of male and {:.2%} of female passengers survived.'.format(classes[i], 
                                                                                        pct_male, pct_female)
    print 'In {} class the ratio of surviving male to female passengers was {:.2f}'.format(classes[i],pct_male/pct_female)

In first class 36.89% of male and 96.81% of female passengers survived.
In first class the ratio of surviving male to female passengers was 0.38
In second class 15.74% of male and 92.11% of female passengers survived.
In second class the ratio of surviving male to female passengers was 0.17
In third class 13.54% of male and 50.00% of female passengers survived.
In third class the ratio of surviving male to female passengers was 0.27


This confirms that the liklihood of dying was definitely higher in the lower classes, but also shows that overall men in the first class were still more likely to survive relative to their female counterparts than those in the other classes. The story told by the numbers in the second class is interesting as it suggests that while survival rates were generally lower than in the first class, men in the second class made the most sacrifices on behalf on the women in their class.

####13.    What else might you conclude?

Without further analysis there doesn't seem to be much else to conclude.

####14. Last, if we were to build a predictive model, which features in the data do you think we should include in the model and which can we leave out? Why?

We can probably leave out ticket number as it is least relevant to conditions aboard the ship. However, if multiple people use the same ticket number, such as families, or if consecutive numbers of the tickets indicate that they were purchased together, then you might expect the ticket numbers to indicate something significant. To incorporate the ticket numbers you could calculate the Hamming distance between all the tickets as a matrix and include the matrix as a set of features during prediction. This could become computationally intractible though with so many tickets (essentially adding that many features to each passenger).

Also, the port of embarkation might not be very significant for the analysis, although it would be difficult to exclude in case cabins were filled by port of embarkation, in which the passengers location within the ship at the time of the sinking might be highly correlated with port of embarkation. I would probably run the analysis both with and without these features.